# Wrangle Report For Udacity Data Wrangling Project

## Table of Contents
<ul>
<li><a href="#gathering">Gathering The Data</a></li>
<li><a href="#assesing">Assessing The Data</a></li>
    <ul>
        <li><a href="#quality">Quality Issues</a></li>
        <li><a href="#tidiness">Tidiness Issues</a></li>
    </ul>  
<li><a href="#cleaning">Cleaning The Data</a></li>
    <ul>
        <li><a href="#missing">Missing Data</a></li>
        <li><a href="#clean_tidiness">Tidiness</a></li>
        <li><a href="#clean_quality">Quality</a></li>
        <li><a href="#clean_tidiness1">Tidiness</a></li>
    </ul>
<li><a href="#save">Save The Results In A Database</a></li>
    <ul>
        <li><a href="#final_df">Final dataframes afther the cleaning part</a></li>
        <li><a href="#tables">Tables Stored In The Database</a></li>   
    </ul>        
</ul>
 

<a id='gathering'></a>
## Gathering The Data

In this project we are going to extract data from different sources related with the Twitter account `@dog_rates`.

Basicly we will work with four data sources:

* **'twitter-archive-enhanced.csv':**  A csv file with 2356 tweets of this account. Each one with a picture of a dog. This file is provided by Udacity for making the project. We read the file and we store it in ha dataframe called `df_twitter_archive_enhanced`.

* **'image-predictions.tsv':** A tsv file with the results obtained of applying a predictive method over the pictures of the tweets. This file was obtained in a project in another nanodegreee and it is provided by Udacity also. Every image in the WeRateDogs Twitter archive was run through a neural network that can classify breeds of dogs. The results: a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images). Udacity gave as the url whre we can find this file, so se have download the file to the local PC and we have store the data into another dataframe called `df_image_predictions`.

* **Additional information obtained with tweetpy:** Once obtained the Twitter credentials, we have uses the tweetpy API to get more additional data. To do this we have connected to the Twitter platform and using tweetpy we have downloaded the tweet status for each tweet in `twitter-archive-enhanced.csv`. Then we have saved these results in a file called `twitter_archive.json` using the json library. Finally, we have read this file and we have extracted some more data to another dataframe called `df_tweepy_extractions` using json again.

* **Information about the replies of each tweet:** Finally we have extracted the data corresponding to the replies for each tweet in `twitter-archive-enhanced.csv`. We have tried some different methods:

    * In some places it is recommended to use tweepy to make a query of all the tweets referenced to @rate_dogs, and search which of them are a reply to the status of the tweet. Translated to code, something like this:
    
            consumer_key = 'XXXXXX'
            consumer_secret = 'XXXXXX'
            access_token = 'XXXXXX'
            access_secret = 'XXXXXX'

            auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
            auth.set_access_token(access_token, access_secret)

            twapi = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
            replies=[]

            for tweet in tweepy.Cursor(twapi.search,q='to:'+name, since_id=892420643555336193, result_type='recent',timeout=999999).items(1000)
                if hasattr(tweet, 'in_reply_to_status_id_str'):
                    if (tweet.in_reply_to_status_id_str==tweet_id):
                        replies.append(tweet)
            
      but it has a lot of limitations and I didn't like it too much.

    * In other places it is recommended to use the urllib3 library to request pages. Then, you can use BeautifulSoup to interpret the result and scrapp the information that you need:
    
             http = urllib3.PoolManager()
             url = "https://twitter.com/dog_rates/status/892420643555336193"
             r = http.request('GET', url)
             soup = BeautifulSoup(r.data)
             tweets = soup.find_all('li','js-stream-item')
             for tweet in tweets:

             full_name = tweet.find("span", "FullNameGroup").find("strong", "fullname").contents[0]  
        
      But, in this case, you need to make scroll down on the page to see all the replies. Even so, when there are too many replies, the page cut the list and ask you in a link if you want to see more. You had to do this as many times as you need until you reach the end of the list. Apart from that, sometimes there are replies to the replies, and the page has another link to select to see them. I mean that with a single request you can't see all the replies if these are a lot.
      Maybe yo can do that using additional requests with POST or some other commands and sending the correct instruction to click in all the necessary links. But I felt like it was too much complicated.
       
    * Finally, I have used the `selenium` library. It permits you to use a local browser to open the pages. In our case we have used FireFox, but it works with Chrome and other navigatos also. You can navigate on these pages programatically using the local navigator and select and click any element inside the page. Once you have deployed completely the page you can get it to a beautifulsoup object and interpret it. Using a local browser can be viewed as a disadvantage, but I felt more comfortable with this method and it is what I have used, but it is a bit slow. I would like to try also the option with urllib3 and use POSTs messages to click in the 'See more' buttons to view all the replies. It could be quicker.




<a id='assesing'></a>
## Assessing The Data

* For assesing the data we have inspect visually each datagrame with the method `sample` and we have also view the data types with the method `info`.


* Specifically in `df_twitter_archive_enhanced` we have seen also that when another user is mentioned in the text, because it is a retweet for example, it appears the use name as @XXXXX. It is not a real problem, but we prefer to delete this parts because we are going to store the mentions, retweets, etc in another dataframe.

* We have seen also that the numbers in the numerator and denominator of the ranks are somtimes suspectful. Taking a look some of them are correct, but others are wrong, because maybe theuy are jokes, for example the number 666 to rate a dog disguised as a devil. Many times also the number in the text uses decimals, so the method used to extract the information only have got the left part of number because it expect an integer.

* We have notice also in this dataframe that the method used to extract the name of the dog is using the word tha follows the sentence 'This is...' (Toby, Ted, etc). Tha means that many times the name extracte is not correct also. I did not think that this was important by the moment, so I did not include this as an issue.

* Analyzing the dog clasification as 'doggo', 'fluffo', etc, I have seen that there are not many dogs clasified and some of them have two clasifications. This variable should be also changed to be stored in a sigle column.

* In `df_image_predictions` I took a look and I did not see any relevant issue apart of some data types.

* In `df_tweepy_extractions` I have seen tha I do not really need the name of the users, because I am going to use their id. The columns related with user mentions, user retweeted, user quoted, etc are stored as lists and in the dataframe appear as a text after reading the json file. This can be problamatic to work with. In any cas we ar going to store all this information in another separeted dataframe to have together all the useres interactions. So this columns will not be necessary.

* In `df_scrapped_replies` we do not need also the name of the user. And we have notice also some data types that need be changed.



As a summary I present below the list of Issues that I have encountered:

<a id='quality'></a>
#### Quality

##### `df_twitter_archive_enhanced` table:

- Some tweets are not available now. 
- The type of the columns `tweet_id`, `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` should be string.
- We are not intereste in the first par of the text (RT @XXXX:) when the tweet is a retweet. I already have this information in other columns.
- There are 23 rows with rating_denominator different to 10.
- And many rows have a numerator not very realistic.
- The name of the dogs `a`, `O`, `by`, `an`, `the`, `his`, `all`and `my` are incorrect. (We are not going to correct this by the moment. We do not need them)
- There are not many dogs classified as doggo, floofer, etc. And 14 of them have double clasification.
- We would like to know the number of replies to each tweet.


##### `df_image_predictions` table:

- The type of the column `tweet_id` should be string.


##### `df_tweepy_extractions` table:

- The column `retweet_count_retweet` has no sense because it has the same value as `retweet_count`.
- we don't need the columns `entities_name`, `entities_screen_name`, `entities_type`, `mentions_name` and `mentions_screen_name` because we are going to use only the ids, so we can drop them by now.
- Nulls represented as void strings in `entities_name`,	`entities_screen_name`, `entities_type`, `entities_user_id`,	`entions_name`,	`mentions_screen_name`,	`mentions_user_id`,	`quoted_status_id`,	`quoted_status_id_rwetweet` and `retweet_count_retweet`.


##### `df_scrapped_replies` table:

- we don't need the columns `user_name` and `full_name` because we are goin to user only the id.
- The type of the columns `favs`, `replies` and `retweets` should be integer instead of float.
- The type of the columns `user_id` and `reply_id` should be a string.
- `language` type should be categorical.
	




<a id='tidiness'></a>
#### Tidiness

##### `df_twitter_archive_enhanced` table:

- Columns `doggo`, `floofer`, `pupper` and `puppo` should be a unique column called dog_type.


##### `df_image_predictions` table:

- this dataframe should be integrated in df_twitter_archive_enhanced.

##### `df_tweepy_extractions` table:

- this dataframe should be integrated in df_twitter_archive_enhanced.

##### `df_scrapped_replies` table:


##### `df_twitter_archive_enhanced` and `df_scrapped_replies` 

- We want mentions, replies, retweets, quoted, etc grouped apart as a different kind of information. We can drop this columns in the original dataframes. We will do this at the end of the clean work.


<a id='cleaning'></a>
## Cleaning The Data

First, we have make a copy of all dataframes to work with.

- df_twitter_archive_enhanced_copy
- df_image_predictions_copy
- df_tweepy_extractions_copy
- df_scrapped_replies_copy

<a id='missing'></a>
### Missing Data

* At this moment there are tweets that are not available on the network. So we delte all the rows in `df_twitter_archive_enhanced_copy` that are not present in the data frame `df_tweepy_extractions_copy` that we have extracted from the network.

* There are not many dogs classified as doggo, floofer, etc. And 14 of them have double clasification. I have checked visually the dogs with double clasification and I have selected the most likely option. In reference to the lack of data I have thought that I can extract some information about that from the replies of the tweet. So I have counted the number of times that doggo, floofer, etc appear in the replies and I have assigned the most frequent value to the dog type, given priority to the current type if it already exists in the table.

* There are 23 rows with rating_denominator different to 10. I have listed all the tweets with a rating_denominator different to 10 and checked visually one by one. In some cases they are correct in other they are not and you can correct them or, sometimes, you do not have a good value to correct the rank and I have filled thes fields with not a number.

* I have done the same with the numerator, but in these case I have reviewed the values greater than 20.

* I was not able to extract the number of replies for each tweet using the API, so I have added a new column to `df_twitter_archive_enhanced_copy` counting the replies estoured in `df_scrapped_replies_copy`

<a id='clean_tidiness'></a>
### Tidiness

* In `df_twitter_archive_enhanced_copy` I have changed the columns `doggo`, `floofer`, `pupper` and `puppo` by a sigle categorical column called `dog_type` with four values.

<a id='clean_quality'></a>
### Quality

* In `df_twitter_archive_enhanced_copy` I have changed the type of the columns `tweet_id`, `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` to String. They are IDs so I thought that it is better a string than an integer, and you can loose information also if you save this data into an excel sheet as integer because some of then are too long.

* In `df_twitter_archive_enhanced_copy` I have dropped all the references to other user names (@XXXXX).

*  I have deleted from `df_tweepy_extractions` the columns `entities_name`, `entities_screen_name`, `entities_type`, `mentions_name` and `mentions_screen_name` because we are going to use only the ids, so we can drop them by now.

*  In `df_tweepy_extractions_copy` The column `retweet_count_retweet` has no sense because it has the same value as `retweet_count`, so we have drop this column.

*  In `df_image_predictions_copy` I have changed the type of the column `tweet_id` to string also.

* In `df_scrapped_replies_copy` I have changed the type of the columns `favs`, `replies` and `retweets` to integer from float.

* In `df_scrapped_replies_copy` the type of the columns `user_id` and `reply_id` has been changed to string also.

* In `df_scrapped_replies` the column `language` has been changed to categorical.

* I have drop the columns `user_name` and `full_name` in `df_scrapped_replies_copy` because we are going to use the id.

<a id='clean_tidiness1'></a>
### Tidiness

* I have returned to tidiness issues because there is a problem that I prefered to arrange it at the end.
I have mereged `df_tweepy_extractions_copy` into `df_twitter_archive_enhanced_copy` because both have information about the tweets in the @rate_dogs account.

* I have mereged `df_image_predictions_copy` into `df_twitter_archive_enhanced_copy` because both have information about the tweets in the @rate_dogs account.

* I said that I preferred to gather all the information related with user mentions, replies to a tweet, retweets, quoted, etc  in `df_twitter_archive_enhanced_copy` and `df_scrapped_replies_copy` summarized in one sigle table. So we could have all the information regarded with interactions between users in a single table.

What we have done is to create a new data frame `df_interactions` whit this information. This data frame has the colums:

- **tweet_id:** Id of the tweet where the interaction between users is located.
- **user_destiny:** User ID who is subject of the interaction: The user mentioned, the user who has been replied, the user whose tweet has been quoted, etc.
- **tweet_id_origin:** Tweet ID of the tweet that has been replied, quoted, etc. If the reply is a reply to another reply, we store the ID of the first original tweet. In this case the ID of the tweet from @rate_dogs that originated the conversation.
- **user_origin:** User ID who is object of the interaction: The user who mentions, the user who replies, the user whose tweet is a quote, etc.
- **interaction_type:** Different types of interactions between users: 'in_reply_to_user_id', 'retweeted_status_user_id',
       'quoted_user_id', 'entities_user_id', 'mentions_user_id'and 'references_to_user'.

For example, if the user A reply to a tweet of the user B. The we add a row with:
- **tweet_id:** Id of the reply.
- **user_destiny:** Id of the replied user.
- **tweet_id_origin:** Id of the replied tweet.
- **user_origin:** Id of the replying user.
- **interaction_type:** 'in_reply_to_user_id'

Or if a user is mentioned in other tweet in the replies:

- **tweet_id:** Id of the tweet with the mention.
- **user_destiny:** Id of the mentioned user.
- **tweet_id_origin:** In this case I have used the original tweet ID from @dog_rates.
- **user_origin:** Id of the mentioning user.
- **interaction_type:** 'references_to_user'

I have created this new dataframe and dropped `'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'entities_user_id', 'mentions_user_id', 'quoted_status_id', 'quoted_user_id', 'quoted_status_id_rwetweet', 'quoted_user_id_retweet'` in `df_twitter_archive_enhanced_copy`.


<a id='save'></a>
## Save The Results In A Database

* Finally we have saved the final dataframes in a local file called `we_rate_dogs.db` using the library sqlite3 for a further analysis.

<a id='final_df'></a>
### Final dataframes afther the cleaning part:

**df_twitter_archive_enhanced_copy**

* **tweet_id (String):** The integer representation of the unique identifier for this Tweet. 
* **timestamp (String):** date and time of the tweet.
* **source (String):** Utility used to post the Tweet, as an HTML-formatted string. Tweets from the Twitter website have a source value of web.
* **text (String):** The actual UTF-8 text of the status update. 
* **expanded_urls (String):** url of the tweet.
* **rating_numerator (Float64):** numerator of the rating assigned according to the text of the tweet.
* **rating_denominator (Float64):** denominator of the rating assigned according to the text of the tweet.
* **name (String):** name of the dog according to the text of the tweet.
* **dog_type (String):** type of the dog acording to the text and to the clasification used in the page:'doggo', 'fluffer', 'pupper' and 'puppo'.
* **replies_count (Int64):** Number of replies to each tweet in @dog_rates account.
* **retweet_count (Float64):** Number of times this Tweet has been retweeted.
* **favorite_count (Float64):** Indicates approximately how many times this Tweet has been liked by Twitter users.
* **favorites_count_retweet (Float64):** This field only surfaces when the Tweet is a retweet. Indicates approximately how many times the original Tweet has been liked by Twitter users. 
* **jpg_url (String):** The url of the image of the tweet. It can be downloaded. 
* **img_num (Int64):** The image with the most confident prediction.
* **p1 (String):** I is the algorithm's #1 prediction for the image in the tweet.
* **p1_conf (Float64):** It is how confident the algorithm is in its #1 prediction.
* **p1_dog (Int64):** It is whether or not the #1 prediction is a breed of dog.
* **p2 (String):** It is the algorithm's second most likely prediction.
* **p2_conf (Float64):** It is how confident the algorithm is in its #2 prediction.
* **p2_dog (Int64):** It is whether or not the #2 prediction is a breed of dog.
* **p3 (String): It is the algorithm's third most likely prediction.
* **p3_conf (Float64):** It is how confident the algorithm is in its #3 prediction.
* **p3_dog (Int64):** It is whether or not the #3 prediction is a breed of dog.

**df_scrapped_replies_copy**

* **conversation (String):** Id of the replied tweet.
* **favs (Int64):** Number of favorites for this replying tweet.
* **image (String):** If there is an image in the reply, it especifies the url.
* **language (String):** When present, indicates a BCP 47 language identifier corresponding to the machine-detected language of the Tweet text.
* **references (String):** Other users ID that are referenced in the text of the reply, if they exist.
* **replies (Int64):** Number of replies to this reply.
* **reply_id (String):** tweet ID for this reply.
* **retweets (Int64):** Number of retweets of this reply.
* **text (String):** Text include in the reply.
* **timestamp (String):** Date_time of the reply.
* **user_id (String):** Id of the user who has replied.


**df_interactions**

* **tweet_id (String):** Id of the tweet where the interaction between users is located.
* **user_destiny (String):** User ID who is subject of the interaction: The user mentioned, the user who has been replied, the user whose tweet has been quoted, etc.
* **tweet_id_origin (String):** Tweet ID of the tweet that has been replied, quoted, etc. If the reply is a reply to another reply, we store the ID of the first original tweet. In this case the ID of the tweet from @rate_dogs that originated the conversation.
* **user_origin (String):** User ID who is object of the interaction: The user who mentions, the user who replies, the user whose tweet is a quote, etc.
* **interaction_type (String):** Different types of interactions between users: 'in_reply_to_user_id', 'retweeted_status_user_id', 'quoted_user_id', 'entities_user_id', 'mentions_user_id'and 'references_to_user'.



<a id='tables'></a>
### Tables Stored In The Database:

This are the name of the tables stores int de DBLite database with the content indicated in the previous dataframes.

* **twitter_master**

* **scrapped_replies**

* **interactions**
